In [24]:
import pandas as pd
import numpy as np

In [2]:
# reading csv file
df_cars = pd.read_csv('Car_sales.csv')

In [3]:
# selecting columns
df_cars = df_cars[['Manufacturer', 'Sales_in_thousands', 'Vehicle_type', 'Price_in_thousands', 'Engine_size', 'Horsepower', 'Fuel_capacity']]

In [4]:
df_cars.head()

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Acura,16.919,Passenger,21.50,1.8,140.0,13.2
1,Acura,39.384,Passenger,28.40,3.2,225.0,17.2
2,Acura,14.114,Passenger,NaN,3.2,225.0,17.2
3,Acura,8.588,Passenger,42.00,3.5,210.0,18.0
4,Audi,20.397,Passenger,23.99,1.8,150.0,16.4


In [5]:
# calculate number of unique values
df_cars.nunique()

Manufacturer           30
Sales_in_thousands    157
Vehicle_type            2
Price_in_thousands    152
Engine_size            31
Horsepower             66
Fuel_capacity          55
dtype: int64

## agg() 

In [6]:
# calculate the sum of columns
df_cars.agg('sum')

Manufacturer          AcuraAcuraAcuraAcuraAudiAudiAudiBMWBMWBMWBuick...
Sales_in_thousands                                             8320.698
Vehicle_type          PassengerPassengerPassengerPassengerPassengerP...
Price_in_thousands                                             4245.567
Engine_size                                                       477.5
Horsepower                                                      29008.0
Fuel_capacity                                                    2800.5
dtype: object

In [8]:
# calculate the mean of numeric columns
df_cars.agg('mean',numeric_only=True)

Sales_in_thousands     52.998076
Price_in_thousands     27.390755
Engine_size             3.060897
Horsepower            185.948718
Fuel_capacity          17.951923
dtype: float64

In [25]:
# calculate the mean and count values
numeric_cols = list(df_cars.select_dtypes(include=np.number).columns)
df_cars[numeric_cols].agg(['mean','count'])   

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
mean,52.998076,27.390755,3.060897,185.948718,17.951923
count,157.000000,155.000000,156.000000,156.000000,156.000000


In [27]:
# Different aggregations per column: calculate the sum and mean of 'Sales_in_thousands' and sum and max of 'Price_in_thousands'
df_cars.agg({'Sales_in_thousands' : ['sum', 'mean'], 'Price_in_thousands' : ['sum', 'max']}).round(1)

,Sales_in_thousands,Price_in_thousands
sum,8320.7,4245.6
mean,53.0,NaN
max,NaN,85.5


In [29]:
# aggregate over the columns.
df_cars[['Sales_in_thousands', 'Price_in_thousands']].agg('sum', axis=1)

0      38.419
1      67.784
2      14.114
3      50.588
4      44.387
        ...  
152    27.945
153    42.745
154    46.331
155    48.993
156    54.969
Length: 157, dtype: float64

In [30]:
# equivalent (NaN behaves differently)
df_cars['Sales_in_thousands'] + df_cars['Price_in_thousands']

0      38.419
1      67.784
2         NaN
3      50.588
4      44.387
        ...  
152    27.945
153    42.745
154    46.331
155    48.993
156    54.969
Length: 157, dtype: float64

In [31]:
# aggregate different functions and rename the index of the resulting DataFrame.
df_cars.agg(x=('Sales_in_thousands', 'sum'), y=('Price_in_thousands', 'sum'))

,Sales_in_thousands,Price_in_thousands
x,8320.698,NaN
y,NaN,4245.567


## The Split-Apply-Combine Strategy

In [32]:
# split data into separate groups
car_filter = df_cars['Vehicle_type']=='Car'
passenger_filter = df_cars['Vehicle_type']=='Passenger'

In [33]:
# apply an operation
car_average = df_cars[car_filter]['Sales_in_thousands'].mean()
passenger_average = df_cars[passenger_filter]['Sales_in_thousands'].mean()

In [34]:
# combine results
pd.DataFrame({'Vehicle_type':['Car', 'Passenger'],
              'Sales_in_thousands':[car_average, passenger_average]}).set_index('Vehicle_type')

,Sales_in_thousands
Vehicle_type,
Car,80.622293
Passenger,43.234345


## groupby()

In [36]:
# group by Vehicle_type and calculate the mean
df_cars.groupby('Vehicle_type').mean(numeric_only=True)

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Vehicle_type,,,,,
Car,80.622293,26.319975,3.520000,186.400000,21.895000
Passenger,43.234345,27.763200,2.902586,185.793103,16.592241


In [41]:
# group by Manufacturer
groupby_obj = df_cars.groupby('Manufacturer')

In [43]:
# the groupby_obj
groupby_obj

In [44]:
# groups attribute and keys
groupby_obj.groups.keys()

dict_keys(['Acura', 'Audi', 'BMW', 'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Dodge', 'Ford', 'Honda', 'Hyundai', 'Infiniti', 'Jaguar', 'Jeep', 'Lexus', 'Lincoln', 'Mercedes-B', 'Mercury', 'Mitsubishi', 'Nissan', 'Oldsmobile', 'Plymouth', 'Pontiac', 'Porsche', 'Saab', 'Saturn', 'Subaru', 'Toyota', 'Volkswagen', 'Volvo'])

In [46]:
# get a specific group within the Manufacturer column
groupby_obj.get_group('Ford').head()

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
46,Ford,70.227,Passenger,12.070,2.0,110.0,12.7
47,Ford,113.369,Passenger,21.560,3.8,190.0,15.7
48,Ford,35.068,Passenger,17.035,2.5,170.0,15.0
49,Ford,245.815,Passenger,17.885,3.0,155.0,16.0
50,Ford,175.670,Passenger,12.315,2.0,107.0,13.2


In [48]:
# group by Manufacturer and calculate the mean
df_cars.groupby('Manufacturer').mean(numeric_only=True).head()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,19.751250,30.633333,2.925000,200.000000,16.400000
Audi,13.519000,39.980000,2.933333,220.000000,19.533333
BMW,15.501667,33.096667,2.700000,185.333333,17.233333
Buick,60.504750,26.781250,3.625000,206.250000,17.750000
Cadillac,22.435600,40.254000,4.500000,256.000000,20.800000


In [49]:
# grou by Vehicle_type and count values
df_cars.groupby('Vehicle_type').count()

,Manufacturer,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Vehicle_type,,,,,,
Car,41,41,40,40,40,40
Passenger,116,116,115,116,116,116


In [50]:
# find null values in columns
df_cars.isnull().sum()

Manufacturer          0
Sales_in_thousands    0
Vehicle_type          0
Price_in_thousands    2
Engine_size           1
Horsepower            1
Fuel_capacity         1
dtype: int64

In [55]:
# group by Engine_size and count values without dropping NaN values
df_cars.groupby('Engine_size', dropna=False).count().tail(5)

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Horsepower,Fuel_capacity
Engine_size,,,,,,
5.2,2,2,2,2,2,2
5.4,1,1,1,1,1,1
5.7,2,2,2,2,2,2
8.0,1,1,1,1,1,1
NaN,1,1,1,0,0,0


## groupby() and agg()

In [56]:
# find the minimum and maximum values on each column
df_cars.groupby('Vehicle_type').agg(['min', 'max'])

Manufacturer         Sales_in_thousands           \
                      min     max                min      max   
Vehicle_type                                                    
Car              Cadillac  Toyota              9.126  540.561   
Passenger           Acura   Volvo              0.110  247.994   

             Price_in_thousands         Engine_size      Horsepower         \
                            min     max         min  max        min    max   
Vehicle_type                                                                 
Car                      11.528  60.105         2.0  5.7      119.0  300.0   
Passenger                 9.235  85.500         1.0  8.0       55.0  450.0   

             Fuel_capacity        
                       min   max  
Vehicle_type                      
Car                   15.1  32.0  
Passenger             10.3  23.7

In [57]:
# set a name for aggregated column
# find the min Engine_size and max Horsepower
df_cars.groupby('Vehicle_type').agg(min_engine_size=('Engine_size', 'min'),
                                    max_horsepower=('Horsepower', 'max'))

,min_engine_size,max_horsepower
Vehicle_type,,
Car,2.0,300.0
Passenger,1.0,450.0


In [58]:
# calculate the sum of Sales_in_thousands and mean of Price_in_thousands
df_cars.groupby('Manufacturer').agg(sum_sales=('Sales_in_thousands','sum'),
                                    mean_price=('Price_in_thousands','mean')).head()

,sum_sales,mean_price
Manufacturer,,
Acura,79.005,30.633333
Audi,40.557,39.980000
BMW,46.505,33.096667
Buick,242.019,26.781250
Cadillac,112.178,40.254000


In [59]:
df_cars.groupby('Manufacturer').sum()['Sales_in_thousands'].head()

Manufacturer
Acura        79.005
Audi         40.557
BMW          46.505
Buick       242.019
Cadillac    112.178
Name: Sales_in_thousands, dtype: float64

In [61]:
df_cars.groupby('Manufacturer').mean(numeric_only=True)['Price_in_thousands'].head().round(2)

Manufacturer
Acura       30.63
Audi        39.98
BMW         33.10
Buick       26.78
Cadillac    40.25
Name: Price_in_thousands, dtype: float64

## Lambda

In [63]:
df_cars.groupby('Manufacturer').sum(numeric_only=True).head()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,79.005,91.900,11.7,800.0,65.6
Audi,40.557,119.940,8.8,660.0,58.6
BMW,46.505,99.290,8.1,556.0,51.7
Buick,242.019,107.125,14.5,825.0,71.0
Cadillac,112.178,201.270,22.5,1280.0,104.0


In [69]:
# get actual sales and price values
df_cars.groupby('Manufacturer').sum()[['Sales_in_thousands', 'Price_in_thousands']].apply(lambda x:x*1000).head()

,Sales_in_thousands,Price_in_thousands
Manufacturer,,
Acura,79005.0,91900.0
Audi,40557.0,119940.0
BMW,46505.0,99290.0
Buick,242019.0,107125.0
Cadillac,112178.0,201270.0


In [76]:
# subtrack values from the mean of each group
df_cars.groupby('Manufacturer').apply(lambda x:x-x.mean(numeric_only=True), include_groups=False)
# [['Price_in_thousands', 'Sales_in_thousands']]

Engine_size  Fuel_capacity  Horsepower  Price_in_thousands  \
Manufacturer                                                                   
Acura        0      -1.125000      -3.200000  -60.000000           -9.133333   
             1       0.275000       0.800000   25.000000           -2.233333   
             2       0.275000       0.800000   25.000000                 NaN   
             3       0.575000       1.600000   10.000000           11.366667   
Audi         4      -1.133333      -3.133333  -70.000000          -15.990000   
...                       ...            ...         ...                 ...   
Volvo        152    -0.400000      -2.033333  -22.166667           -6.533333   
             153     0.100000       0.066667  -14.166667           -3.433333   
             154     0.100000       0.066667  -14.166667           -2.133333   
             155     0.000000       0.666667   53.833333           14.566667   
             156     0.600000       3.266667   18.833333            5.066667   

                  Sales_in_thousands Vehicle_type  
Manufacturer                                       
Acura        0             -2.832250          NaN  
             1             19.632750          NaN  
             2             -5.637250          NaN  
             3            -11.163250          NaN  
Audi         4              6.878000          NaN  
...                              ...          ...  
Volvo        152           -9.078333          NaN  
             153            2.621667          NaN  
             154            4.907667          NaN  
             155           -9.130333          NaN  
             156            6.345667          NaN  

[157 rows x 6 columns]

In [78]:
df_cars.head()

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Acura,16.919,Passenger,21.50,1.8,140.0,13.2
1,Acura,39.384,Passenger,28.40,3.2,225.0,17.2
2,Acura,14.114,Passenger,NaN,3.2,225.0,17.2
3,Acura,8.588,Passenger,42.00,3.5,210.0,18.0
4,Audi,20.397,Passenger,23.99,1.8,150.0,16.4


In [80]:
df_cars.groupby('Manufacturer').mean(numeric_only=True).head()

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,19.751250,30.633333,2.925000,200.000000,16.400000
Audi,13.519000,39.980000,2.933333,220.000000,19.533333
BMW,15.501667,33.096667,2.700000,185.333333,17.233333
Buick,60.504750,26.781250,3.625000,206.250000,17.750000
Cadillac,22.435600,40.254000,4.500000,256.000000,20.800000


## Filtration

In [82]:
# calculate the mean of Sales_in_thousands
df_cars.mean(numeric_only=True)

Sales_in_thousands     52.998076
Price_in_thousands     27.390755
Engine_size             3.060897
Horsepower            185.948718
Fuel_capacity          17.951923
dtype: float64

In [83]:
# filter certain values out based on sum() and return a subset
def filter_func(x):
    return x['Sales_in_thousands'].sum() > 52

df_filter = df_cars.groupby(['Manufacturer']).filter(filter_func)
df_filter

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Acura,16.919,Passenger,21.500,1.8,140.0,13.2
1,Acura,39.384,Passenger,28.400,3.2,225.0,17.2
2,Acura,14.114,Passenger,NaN,3.2,225.0,17.2
3,Acura,8.588,Passenger,42.000,3.5,210.0,18.0
10,Buick,91.561,Passenger,21.975,3.1,175.0,17.5
...,...,...,...,...,...,...,...
152,Volvo,3.545,Passenger,24.400,1.9,160.0,15.8
153,Volvo,15.245,Passenger,27.500,2.4,168.0,17.9
154,Volvo,17.531,Passenger,28.800,2.4,168.0,17.9
155,Volvo,3.493,Passenger,45.500,2.3,236.0,18.5


In [84]:
df_cars.groupby(['Manufacturer']).sum(numeric_only=True)['Sales_in_thousands'].sort_values().head(7)

Manufacturer
Porsche     12.128
Jaguar      15.467
Saab        21.306
Infiniti    23.713
Audi        40.557
BMW         46.505
Plymouth    64.001
Name: Sales_in_thousands, dtype: float64

In [85]:
df_cars[df_cars['Manufacturer'].isin(['BMW','Audi','Infiniti','Saab','Jaguar','Porsche'])].shape

(13, 7)

In [86]:
print(df_cars.shape)
print(df_filter.shape)

(157, 7)
(144, 7)
